# Исследование надёжности заёмщиков

**Цель исследования** — проверка следующих гипотез: 
- количество детей влияет на факт погашения кредита в срок
- семейное положение влияет на факт погашения кредита в срок
- уровень дохода влияет на факт погашения кредита в срок
- цель кредита влияет на факт его погашения в срок

**Входные данные** представляют собой статистику о платежеспобобности клиентов банка.

## Шаг 1. Открытие файла с данными и ознакомление с ним

In [1]:
# import pandas library
import pandas as pd

In [2]:
# open file
data = pd.read_csv('/Users/aleksandrdudakov/Downloads/data.csv') 

In [3]:
# show first 15 rows
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info() # general information about data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<a id='review'></a>

In [5]:
# values distribution for each column
for value in data:
    print(data[value].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, d

Опишем замеченные проблемы в столбцах, если они есть:
- `children` — количество детей в семье (*аномальные значения -1 и 20*)
- `days_employed` — общий трудовой стаж в днях (*в столбце есть пропуски, значения не целые и есть отрицательные*)
- `dob_years` — возраст клиента в годах (*101 нулевое значение*)
- `education` — уровень образования клиента (*дубликаты: значения в разных строках написаны в разных регистрах*)
- `education_id` — идентификатор уровня образования (зависит от значения в предыдущем столбце)
- `family_status` — семейное положение (*стилистическая ошибка: не соблюден регистр*)
- `family_status_id` — идентификатор семейного положения (зависит от значения в предыдущем столбце)
- `gender` — пол клиента (*аномальное значение XNA*)
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов (принимает значения 0 или 1)
- `total_income` — ежемесячный доход (*в столбце есть пропуски*)
- `purpose` — цель получения кредита (*возможны дубликаты*)

## Шаг 2. Предобработка данных

### 2.1 Обработка пропусков

В столбцах `days_employed` и `total_income` мы обнаружили пропущенные значения, разберемся с ними. Посмотрим часть строк с пропущенными значеними.

In [6]:
# first 5 rows of missing values in column 'days_employed'
data[data['days_employed'].isna()].head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [7]:
# first 5 rows of missing values in column 'total_income'
data[data['total_income'].isna()].head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Похоже, что пропущенные значения в этих двух столбцах находятся в одних и тех же строках. Проверим это: выведем таблицу, где есть пропуски в обоих столбцах.

In [8]:
# rows where there are gaps in columns 'total_income' and 'days_employed'
data[data['total_income'].isna() & data['days_employed'].isna()] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Получили таблицу с 2174 строками. Теперь найдем количество пропусков в обоих столбцах.

In [9]:
# number of missing values in column 'days_employed'
len(data[data['days_employed'].isna()]) 

2174

In [10]:
# number of missing values in column 'total_income'
len(data[data['total_income'].isna()]) 

2174

Количество совпадает. Значит, пропуски в столбцах `total_income` и `days_employed` всегда идут вместе.

Найдем долю пропусков в этих столбцах.

In [11]:
ratio = len(data[data['days_employed'].isna()]) / len(data)
print(f'Доля пропусков в столбцах "total_income" и "days_employed": {ratio:.0%}')

Доля пропусков в столбцах "total_income" и "days_employed": 10%


10% - это довольно много. Возможно, пропуски сделаны намеренно (например, для чиновников). Посмотрим, как пропуски распределены по типу занятости.

In [12]:
print(data[data['days_employed'].isna()]['income_type'].value_counts()) # distribution of missing values by income type

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Теперь посмотрим распределение типа занятости по всем данным.

In [13]:
# distribution of all values by income type
print(data['income_type'].value_counts()) 

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


Видно, что пропуски распредены равномерно по типу занятости (и пропуски составляют около 10% значения для всех типов занятости, где больше двух значений): наша гипотеза не подтвердилась. 

Получается, что пропуски случайны, и их можно заменить медианным значением (оно лучше, чем среднее, подходит для таких количественных переменных как доход или стаж, так как медиана менее чувствительна к выдающимся значениям в выборке). Найдем медианные значения столбцов `days_employed` и `total_income` и заменим ими пропущенные значения.

In [14]:
# replacing missing values with the median
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median()) 
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

Убедимся, что пропущенных значений больше нет.

In [15]:
# general information about data
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


#### Вывод

В столбцах `days_employed` и `total_income` были обнаружены пропущенные значения. Мы убедились, что они случайно распределены по типу занятости, и заменили их медианным значением по столбцу.

### 2.2 Обработка аномалий

Сначала рассмотрим столбец `days_employed`. Значения в нем не целые, а также есть отрицательные. Найдем минимальное и максимальное значения в столбце.

In [16]:
print(f'Максимум, минимум в столбце "days_employed": {data["days_employed"].max()}, {data["days_employed"].min()}')
print(f'Количество нулевых значений в столбце "days_employed": {len(data[data["days_employed"] == 0])}')

Максимум, минимум в столбце "days_employed": 401755.40047533, -18388.949900568383
Количество нулевых значений в столбце "days_employed": 0


Получается, в столбце есть отрицательные и положительные значения. Посмотрим распределение отрицательных и положительных значений по типу занятости (нулевых нет).

In [17]:
# distribution of positive values by income type
data[data['days_employed'] > 0]['income_type'].value_counts() 

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [18]:
# distribution of negative values by income type
data[data['days_employed'] < 0]['income_type'].value_counts() 

сотрудник          11119
компаньон           5085
госслужащий         1459
пенсионер            413
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Видим, что все безработные и большинство пенсионеров имеют положительный стаж, а все остальные – отрицательный.

Исследуем данные с теми пенсионерами, у которых стаж отрицательный.

In [19]:
# first 5 values of required data
data[(data['income_type'] == 'пенсионер') & (data['days_employed'] < 0)].head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,-1203.369529,65,среднее,1,гражданский брак,1,M,пенсионер,0,145017.937533,сыграть свадьбу
29,0,-1203.369529,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,145017.937533,строительство жилой недвижимости
55,0,-1203.369529,54,среднее,1,гражданский брак,1,F,пенсионер,1,145017.937533,сыграть свадьбу
67,0,-1203.369529,52,высшее,0,женат / замужем,0,F,пенсионер,0,145017.937533,покупка жилья для семьи
145,0,-1203.369529,62,среднее,1,женат / замужем,0,M,пенсионер,0,145017.937533,строительство недвижимости


Похоже, что у них всех одинаковый стаж и ежемесячный доход. Значит, это те пенсионеры, у которых изначально в стобцах `days_employed` `total_income` были пропуски, так как в изначальном датафрейме повторяющихся значений в этих столбцах [не было](#review).

Оценим адекватность положительных значений в `days_employed`. Для этого расчитаем минимум и максимум в годах (поделив на 365).

In [20]:
print(f"Максимум, минимум в столбце 'days_employed' для положительных значений (в годах): {data[data['days_employed'] > 0]['days_employed'].max() / 365}, {data[data['days_employed'] > 0]['days_employed'].min() / 365}")

Максимум, минимум в столбце 'days_employed' для положительных значений (в годах): 1100.6997273296713, 900.6266317932007


Это неадекватное значение. Возможно для положительных значений стаж указан в часах. Поделим все максимум и минимум на 24.

In [21]:
print(f"Максимум, минимум в столбце 'days_employed' для положительных значений (в годах): {data[data['days_employed'] > 0]['days_employed'].max() / 365 / 24}, {data[data['days_employed'] > 0]['days_employed'].min() / 365 / 24}")

Максимум, минимум в столбце 'days_employed' для положительных значений (в годах): 45.8624886387363, 37.526109658050025


Значения между 37 и 45 годами для трудового стажа пенсионера вполне адекватны. Значит, положительные значения значения столбца `days_employed` надо поделить на 24.

In [22]:
def hours_to_days(n):
    '''
    Сonverts number of hours to number of days.
    '''
    return n / 24
data.loc[data['days_employed'] > 0,'days_employed'] = data.loc[data['days_employed'] > 0, 'days_employed'].apply(hours_to_days) # apply convertation to 'days_employed' column

Проверим результат.

In [23]:
print(f"Максимум, минимум в столбце 'days_employed' для положительных значений (в годах): {data[data['days_employed'] > 0]['days_employed'].max() / 365}, {data[data['days_employed'] > 0]['days_employed'].min() / 365}")

Максимум, минимум в столбце 'days_employed' для положительных значений (в годах): 45.8624886387363, 37.526109658050025


Теперь изучим данные с отрицательным стажем. Расчитаем минимум и максимум в годах.

In [24]:
print(f"Максимум, минимум в столбце 'days_employed' для отрицательных значений (в годах): {data[data['days_employed'] < 0]['days_employed'].max() / 365}, {data[data['days_employed'] < 0]['days_employed'].min() / 365}")

Максимум, минимум в столбце 'days_employed' для отрицательных значений (в годах): -0.06614146093282515, -50.38068465909146


Значения попадают в адекватный диапазон, если их взять их по модулю.

In [25]:
data.loc[data['days_employed'] < 0, 'days_employed'] = data.loc[data['days_employed'] < 0, 'days_employed'].apply(abs)

Проверим результат. Найдем длину датафрейма с нулевыми значениями в столбце `days_employed`.

In [26]:
# find lenght of dataframe with negative values of 'days_employed' column
len(data[data['days_employed'] < 0])

0

Длина равна нулю. Аномалию со столбцом `days_employed` можно объяснить, например, ошибкой при совмещении файлов с безработными и пенсионерами и с остальными. Также ошибка могла произойти при переводе стажа в дни.

Перейдем к столбцу `children`. В нем есть аномальные значения `-1` и `20`. `-1` заменим на `1`, а `20` – на `2`.

In [27]:
# replace required values
data.loc[data['children'] == -1, 'children'] = 1 
data.loc[data['children'] == 20, 'children'] = 2

Проверим результат.

In [28]:
# distribution of values in 'children' column
data['children'].value_counts() 

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Аномальных значений нет. Появление этих значений можно объяснить ошибкой при наборе возраста сотрудником или клиентом банка.

В столбце `dob_years` есть 101 нулевых значений. Заменим их средним арифметическим (для возраста среднее подходит лучше, так как возраст распределен более равномерно, чем, например, доход).

In [29]:
# replace zero with average age
mean_age = int(data.loc[data['dob_years'] != 0, 'dob_years'].median())
data.loc[data['dob_years'] == 0, 'dob_years'] = data.loc[data['dob_years'] == 0, 'dob_years'].replace(0, mean_age)

Проверим результат.

In [30]:
# distribution of values in 'dob_years' column
data['dob_years'].value_counts() 

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Нулей нет. Эти нулевые значения можно объяснить тем, что клиент не указал возраст, или тем, что при выгрузке произошла ошибка.

Теперь рассмотрим столбец `gender`: в нем есть аномальное значение `'XNA'`.

In [31]:
# row with anomalous value
data[data['gender'] == 'XNA'] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Присвоим значению этой строки в столбце `gender` моду этого столбца.

In [32]:
# replace 'XNA' with mode of 'gender' column
mode_gender = data['gender'].mode()
data.loc[data['gender'] == 'XNA', 'gender'] = mode_gender

Проверим результат.

In [33]:
# distribution of values in 'gender' column
data['gender'].value_counts() 

F    14236
M     7288
Name: gender, dtype: int64

Осталось только два значения. Третье значение можно объяснить указанием клиентом другого пола или ошибкой при выгрузке.

### Замена типа данных

Изменим тип данных столбцов `total_income` и `days_employed` c `float` на `int`.

In [34]:
# change type of 'total_income' and 'days_employed' columns to 'int'
data['total_income'] = data['total_income'].astype('int') 
data['days_employed'] = data['days_employed'].astype('int')

Проверим.

In [35]:
data.info() # general information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Вывод

Были обработаны аномальные значения в столбцах `children`, `days_employed`, `dob_years`, `gender`.

 ### Обработка дубликатов

Сначала устраним неявные дубликаты в столбце `education`. Для их устранения приведем все значения в этом столбце к нижнему регистру.

~Заодно таким же образом исправим стилистическую ошибку в столбце `family_status`~

In [36]:
# lower all values in 'education' and 'family_status' columns
data['education'] = data['education'].str.lower() 
data['family_status'] = data['family_status'].str.lower() 

Проверим.

In [37]:
# distribution of values in 'education' column
data['education'].value_counts() 

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Неявные дубликаты в столбце `education` устранены. Эти дубликаты можно объяснить челоческим фактором.

Теперь посчитаем количество явных дубликатов в датафрейме.

In [38]:
# display number of duplicates
data.duplicated().sum() 

71

Удалим 71 явных дубликатов.

In [39]:
# drop duplicates and reset index in dataframe
data = data.drop_duplicates().reset_index(drop=True) 

Проверим.

In [40]:
# display number of duplicates
data.duplicated().sum() 

0

Получили 0 явных дубликатов.

Проверим неявные дубликаты в столбце `purpose`.

In [41]:
# distribution of values in 'purpose' column
data['purpose'].value_counts() 

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Видно, что, на самом деле, у кредитов в нашем датафрейме всего 4 цели: проведение свадьбы, покупка недвижимости, покупка автомобиля, получение образования. Сведем все значения столбца `purpose` к четырем основным и создадим на их основе столбец `purpose_category`

In [42]:
data['purpose_category'] = data['purpose']

In [43]:
def find_svadb_in_data(data):
    '''
    Searches for 'свадьб' in string.
    '''
    if 'свадьб' in data:
        return True
    return False

data.loc[data['purpose_category'].apply(find_svadb_in_data), 'purpose_category'] = 'проведение свадьбы' # replaces values in 'purpose_category' column for required strings

In [44]:
def find_nedvizh_in_data(data):
    '''
    Searches for 'жил' or 'недвижим' in string.
    '''
    if 'жил' in data or 'недвижим' in data:
        return True
    return False

data.loc[data['purpose_category'].apply(find_nedvizh_in_data), 'purpose_category'] = 'операции с недвижимостью' # replaces values in 'purpose_category' column for required strings

In [45]:
def find_avto_in_data(data):
    '''
    Searches for 'авто' in string.
    '''
    if 'авто' in data:
        return True
    return False

data.loc[data['purpose_category'].apply(find_avto_in_data), 'purpose_category'] = 'операции с автомобилем' # replaces values in 'purpose_category' column for required strings

In [46]:
def find_obrazov_in_data(data):
    '''
    Searches for 'авто' in string.
    '''
    if 'образов' in data:
        return True
    return False

data.loc[data['purpose_category'].apply(find_obrazov_in_data), 'purpose_category'] = 'получение образования' # replaces values in 'purpose_category' column for required strings

Проверим.

In [47]:
# distribution of values in 'purpose_category' column
data['purpose_category'].value_counts() 

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Осталось только четыри цели кредита. Такое большое количество первоначальных целей кредита можно объяснить тем, что клиенты в свободной форме могли описать цель кредита.

В остальных столбцах дубликатов нет (см. [выше](#review)).

#### Вывод

Были устранены явные дубликаты во всем датафрейме, неявные дубликаты – в столбце `education`, а также создан новый столбец `purpose_category` для устранения "дубликатов" в `purpose`.

### Категоризация данных

Создадим два новых датафрейма со столбцами:
- `education_id` и `education`,
- `family_status_id` и `family_status`.

Эти датафреймы будут использоваться как словари.

In [48]:
# create new dataframe: dictionary 'education_id' to 'education'
education_log = pd.DataFrame(data={
    'education_id': data['education_id'].unique(), 
    'education': data['education'].unique()
})
                             

In [49]:
# create new dataframe: dictionary 'family_status_id' to 'family_status'
family_status_log = pd.DataFrame(data={
    'family_status_id': data['family_status_id'].unique(), 
    'family_status': data['family_status'].unique()
}) 

Проверим.

In [50]:
education_log

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [51]:
family_status_log

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Датафреймы созданы. Теперь удалим стобцы `education` и `family_status` из старого датафрейма.

In [52]:
data = data.drop(['education', 'family_status'], axis=1) # drop columns 'education', 'family_status'

Проверим.

In [53]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose', 'purpose_category'],
      dtype='object')

Столбцы удалены.

Разделим ежемесячный доход на 5 категорий. Для этого создадим новый столбец `total_income_category` и функцию, которую применим к столбцу `total_income` для нахождения категории.

In [54]:
def find_income_category(total_income):
    '''
    Find category for given income.
    '''
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'

data['total_income_category'] = data['total_income'].apply(find_income_category) # create 'total_income_category' and find category for each income

Проверим.

In [55]:
# first 5 rows of dataframe
data.head() 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,получение образования,B
4,0,14177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,C


Столбец `total_income_category` добавлен.

#### Вывод

Столбец `total_income` категоризирован, и на его основе создан новый стобец `total_income_category`.

## Шаг 3. Интерпретация данных

### Влияние количества детей на факт погашения кредита

Проверим первую гипотезу о влиянии количества детей на факт погашения кредита в срок. Для этого посмотрим долю тех, кто имел задолженность для каждого количества детей.

In [56]:
children_pivot = data.pivot_table(index='children', columns='debt', values='dob_years', aggfunc=['count']) # create column
children_pivot['ratio'] = children_pivot['count'][1] / (children_pivot['count'][0] + children_pivot['count'][1]) # create  'ratio' column
children_pivot.sort_values('ratio')

count             ratio
debt            0       1          
children                           
0         13028.0  1063.0  0.075438
3           303.0    27.0  0.081818
1          4410.0   445.0  0.091658
2          1926.0   202.0  0.094925
4            37.0     4.0  0.097561
5             9.0     NaN       NaN

#### Вывод

Заметим, что количество детей не сильно влияет на факт погашение кредита в срок, но бездетные чуть реже имеют задолженность (это можно объяснить тем, что они не тратят деньги на детей). Для клиентов с пятью детьми выборка нерепрезентативная.

### Влияние семейного положения на факт погашения кредита

Проведем такое же исследование для второй гипотезы.

In [57]:
import warnings
warnings.filterwarnings('ignore')
family_status_pivot = data.pivot_table(index='family_status_id', columns='debt', values='dob_years', aggfunc=['count']) # create table
family_status_pivot['ratio'] = family_status_pivot['count'][1] / (family_status_pivot['count'][0] + family_status_pivot['count'][1]) # create 'ratio' column
family_status_pivot.merge(family_status_log, on='family_status_id', how='left') # merge tables 'family_status_log' and 'family_status_pivot'

,family_status_id,"(count, 0)","(count, 1)","(ratio, )",family_status
0,0,11408,931,0.075452,женат / замужем
1,1,3763,388,0.093471,гражданский брак
2,2,896,63,0.065693,вдовец / вдова
3,3,1110,85,0.071130,в разводе
4,4,2536,274,0.097509,не женат / не замужем


#### Вывод

Семейное положение влияет на факт погашения кредита: самые низкая доля у вдовцов и разведенных, самая высокая – у неженатых.

### Влияние уровня дохода на факт погашения кредита

Исследуем влияние уровня дохода на факт погашения кредита.

In [58]:
total_income_pivot = data.pivot_table(index='total_income_category', columns='debt', values='dob_years', aggfunc=['count']) # create table
total_income_pivot['ratio'] = total_income_pivot['count'][1] / (total_income_pivot['count'][0] + total_income_pivot['count'][1]) # create 'ratio' column
total_income_pivot.sort_values('ratio') # display table

count           ratio
debt                       0     1          
total_income_category                       
D                        329    21  0.060000
B                       4685   356  0.070621
A                         23     2  0.080000
C                      14656  1360  0.084915
E                         20     2  0.090909

#### Вывод

Строгой зависимости уровня дохода от факта погашения кредита нет, но, как и ожидалось, клиенты с самым низким уровнем дохода чаще всех имеют задолженности.

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b> Логично👍</div>

## Влияние цели кредита на факт погашения кредита

Изучим, как цель кредита влияет на факт его возврата в срок.

In [59]:
purpose_pivot = data.pivot_table(index='purpose_category', columns='debt', values='dob_years', aggfunc=['count']) # create table
purpose_pivot['ratio'] = purpose_pivot['count'][1] / (purpose_pivot['count'][0] + purpose_pivot['count'][1]) # create 'ratio' column
purpose_pivot.sort_values('ratio') # display table

count          ratio
debt                          0    1          
purpose_category                              
операции с недвижимостью  10029  782  0.072334
проведение свадьбы         2138  186  0.080034
получение образования      3643  370  0.092200
операции с автомобилем     3903  403  0.093590

#### Вывод

Заметим, что наименьшая доля задолженностей у операций с недвижимостью и свадеб, а наибольшая – у получения образования и операций с автомобилем. Это можно объяснить тем, что свадьба оплачивается деньгами, подаренными на нее, или тем, что часто родители помогают расплатиться с ней, а кредиты на операции с недвижимость часто совершают корпоративные клиенты.

# Шаг 4. Общий вывод

Была проведена данного датафрейме, в котором было много ошибок и аномалий: пропуски, неверный тип данных, дубликаты, также была проведена категоризация уровня дохода и целей кредита. 
На основании предобработанного датафрейма были проверены гипотезы и сделаны следующие выводы:
- количество детей *слабо* влияет на факт погашения кредита в срок, но бездетные клиенты чаще, чем другие выплачивают его вовремя;
- семейное положение *влияет* на факт погашения кредита в срок: чаще всего имеют задолженности неженатые, реже всего – вдовцы;
- уровень дохода в общем случае *не влияет* на факт погашения кредита в срок, но клиенты с самым низким уровнем дохода чаще, чем остальные категории клиентов имеют задолженности;
- цель кредита следующим образом влияет на факт его погашения в срок: наименьшая доля задолженностей у клиентов с целью на операции с недвижимостью и проведение свадеб, а наибольшая – у получения образования и операций с автомобилем.
